https://github.com/opencv/opencv/tree/master/data/haarcascades

https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html


https://docs.opencv.org/2.4/doc/user_guide/ug_traincascade.html

In [1]:
import numpy as np
import cv2 as cv

rostro = cv.CascadeClassifier('./cascade.xml')

cap = cv.VideoCapture(0)
x=y=w=h= 0 
img = 0
count = 0
while True:
    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    rostros = rostro.detectMultiScale(gray, 5, 10)
    for(x, y, w, h) in rostros:
        m= int(h/2)
        frame = cv.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)

    cv.imshow('rostros', frame)
    
    k = cv.waitKey(1)
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()
    

KeyboardInterrupt: 

In [ ]:

pwd

In [ ]:
import cv2 as cv

rostro = cv.CascadeClassifier('./cascade.xml')
image_path = './8.jpg'
image = cv.imread(image_path)
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

# Utiliza el clasificador en cascada en la imagen
faces = rostro.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Dibuja los rectángulos alrededor de los rostros detectados
for (x, y, w, h) in faces:
    cv.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Muestra la imagen con las detecciones
cv.imshow('Detecciones en Imagen Estática', image)
cv.waitKey(0)
cv.destroyAllWindows()


In [2]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("./model2-011.model/")

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
        if label == 1:
            cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)

    cv2.imshow('rostros',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()

1/1 [==============================] - 0s 21ms/step


In [ ]:
import cv2
import numpy as np
from keras.models import load_model

model = load_model("./model2-011.model/")

results = {0: 'without mask', 1: 'mask'}
GR_dict = {0: (0, 0, 255), 1: (0, 255, 0)}

rect_size = 4
cap = cv2.VideoCapture(0)

haarcascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im = cv2.flip(im, 1, 1)

    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    
    face_detected = False  # Flag para indicar si se detectó una cara con mascarilla
    
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f]

        face_img = im[y:y+h, x:x+w]
        rerect_sized = cv2.resize(face_img, (150, 150))
        normalized = rerect_sized / 255.0
        reshaped = np.reshape(normalized, (1, 150, 150, 3))
        reshaped = np.vstack([reshaped])
        result = model.predict(reshaped)

        label = np.argmax(result, axis=1)[0]

        if label == 1:  # Si se detecta una cara con mascarilla, activa el frame
            face_detected = True

        cv2.rectangle(im, (x, y), (x+w, y+h), GR_dict[label], 2)
        cv2.rectangle(im, (x, y-40), (x+w, y), GR_dict[label], -1)
        cv2.putText(im, results[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    if face_detected:
        cv2.imshow('rostros', im)
    else:
        # Si no se detecta una cara con mascarilla, desactiva el frame
        cv2.imshow('rostros', np.zeros_like(im))

    key = cv2.waitKey(10)

    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()
